In [1]:
from datetime import datetime, timedelta
import mysql.connector

# Define the start and end dates
start_date = datetime(2018, 1, 1)
end_date = datetime.today()

# Create a list of dates
date_list = []

# Iterate from start_date to end_date and add each date to the list
current_date = start_date
while current_date <= end_date:
    date_list.append((current_date,))
    current_date += timedelta(days=1)

In [2]:
host = "finance.cvm2aaxkmu43.us-east-2.rds.amazonaws.com"
user = "admin"
password = "Nk7f4mJr6?A"
database = "finance"

conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = conn.cursor()

### Insert NU price before IPO

In [3]:
nu_prices = [
        ("NU", "2020-04-01", 1.7354,  1.7354),
        ("NU", "2020-07-01", 1.7883,  1.7883),
        ("NU", "2020-10-01", 1.7883,  1.7883),
        ("NU", "2021-01-01", 3.9783,  3.9783),
        ("NU", "2021-04-01", 3.9783,  3.9783),
        ("NU", "2021-07-01", 4.5283,  4.5283),
        ("NU", "2021-10-01", 4.5283,  4.5283)]

In [4]:
cursor.executemany(
    """
    INSERT IGNORE INTO asset_price(ticker, quote_date, open_price, close_price)
    VALUES(%s, %s, %s, %s)
    """, nu_prices)

### Insert dates

In [5]:
cursor.executemany(
    "INSERT IGNORE INTO dates(date) VALUES(%s)",
    date_list
)

### Insert daily_asset_price

In [6]:
cursor.execute("DELETE FROM daily_asset_price")

In [7]:
cursor.execute(
    """
    INSERT IGNORE INTO daily_asset_price(ticker, date, price)
    WITH date_range AS
    (
        SELECT
            ticker,
            quote_date,
            LEAD(quote_date) OVER(PARTITION BY ticker ORDER BY quote_date) as next_date,
            close_price
        FROM asset_price ap
    )
    SELECT
        ticker,
        COALESCE(d.date, quote_date) as date,
        close_price as price
    FROM date_range dr
    LEFT JOIN dates d
    ON d.date >= dr.quote_date AND d.date < dr.next_date
    """
)

### Insert variable_income_daily_balance

In [8]:
cursor.execute("DELETE FROM variable_income_daily_balance")

In [9]:
cursor.execute(
    """
    INSERT INTO variable_income_daily_balance
    WITH dolar AS(
        SELECT date, price as dolar_price
        FROM daily_asset_price
        WHERE ticker = "BRL=X"
    ),
    currency AS (
        SELECT ticker, MAX(currency) as currency
        FROM variable_income_operations
        GROUP BY ticker
    ),
    operation AS (
        SELECT 	
            ticker,
            currency,
            operation_date,
            LEAD(operation_date) OVER (PARTITION BY ticker ORDER BY operation_date) AS next_operation_date,
            SUM(CASE WHEN operation_type = "buy" THEN amount else amount * -1 END) AS amount
        FROM variable_income_operations o
        GROUP BY ticker, currency, operation_date, operation_type
    ),
    balance AS (
        SELECT
            p.ticker,
            p.date,
            p.price,
            dolar_price,
            c.currency, 
            amount as amount_change,
            SUM(amount) OVER(PARTITION BY ticker ORDER BY p.date) AS amount
        FROM daily_asset_price p
        LEFT JOIN operation o
        ON p.ticker = o.ticker AND p.date = o.operation_date
        LEFT JOIN dolar d
        ON d.date = p.date
        INNER JOIN currency c
        ON c.ticker = p.ticker 
    )
    SELECT
        ticker,
        date,
        price,
        dolar_price,
        currency,
        amount_change,
        amount,
        CASE WHEN currency = "dolar" THEN (price * dolar_price) * amount ELSE price * amount END as value
    FROM balance
    WHERE amount <> 0;
    """
)

### Insert fixed_income_daily_balance

In [10]:
cursor.execute("DELETE FROM fixed_income_daily_balance")

In [11]:
cursor.execute(
    """
     INSERT INTO fixed_income_daily_balance
     WITH base_ipca AS(
            SELECT 
                financial_index,
                date,
                COALESCE(lead(date) OVER (PARTITION BY financial_index ORDER BY date), "2099-01-01") AS next_date,
            factor
        FROM index_series
        WHERE financial_index = "ipca"
    ), daily_indexes AS(
        SELECT 
            ipca.financial_index,
            i.date,
            1 + ipca.factor/100/21 AS factor
        FROM index_series i
        INNER JOIN base_ipca ipca
        ON i.date >= ipca.date and i.date < ipca.next_date
        WHERE i.financial_index = "cdi"
        UNION
        SELECT financial_index, date, factor
        FROM index_series 
        WHERE financial_index <> "ipca"
    ), daily_rates AS (
        SELECT 
            i.date,
            i.factor,
            op.asset,
            op.purchase_date,
            op.value,
            op.pre_rate,
            op.post_rate,
            op.due_date,
            op.tax_rate,
            op.is_pgbl,
            pow((1.0 + pre_rate),(1.0/252)) -1 as daily_pre_rate,
            (op.post_rate * (i.factor - 1) + 1) as daily_post_rate,
            pow((1.0 + pre_rate),(1.0/252)) -1 + (op.post_rate * (i.factor - 1) + 1) as total_daily_rate,
            EXP(SUM(LN(pow((1.0 + pre_rate),(1.0/252)) -1  + (op.post_rate * (i.factor - 1) + 1))) OVER (PARTITION BY op.asset, op.purchase_date ORDER BY i.date)) AS cumulative_daily_rate
        FROM fixed_income_operations op
        INNER JOIN daily_indexes i
        ON op.financial_index = i.financial_index
        AND i.date BETWEEN op.purchase_date AND op.due_date
        WHERE op.asset NOT LIKE "%Tesouro Selic%"
    ), grouped AS (
        SELECT 
            asset,
            due_date,
            date,
            tax_rate,
            SUM(value) as deposit_value,
            SUM(cumulative_daily_rate * value) AS gross_value,
            SUM(CASE WHEN is_pgbl THEN cumulative_daily_rate * value * tax_rate ELSE (cumulative_daily_rate * value - value) * tax_rate END) AS tax_value,
            SUM(cumulative_daily_rate * value - CASE WHEN is_pgbl THEN cumulative_daily_rate * value * tax_rate ELSE (cumulative_daily_rate * value - value) * tax_rate END) AS net_value
        FROM daily_rates
        GROUP BY asset, date, due_date, tax_rate
    ), values_range AS(
         SELECT 
            asset,
            due_date,
            date,
            COALESCE (lead(date) OVER (PARTITION BY asset ORDER BY date), due_date) next_date,
            tax_rate,
            deposit_value,
            gross_value,
            tax_value,
            net_value
        FROM grouped
    )
    SELECT r.asset, r.due_date, d.date, r.tax_rate, r.deposit_value, r.gross_value, r.tax_value, r.net_value
    FROM dates d
    INNER JOIN values_range r
    ON d.date >= r.date and d.date < next_date 
    ORDER BY d.date DESC
    """
)

In [12]:
cursor.execute("""
    WITH operations AS (
        SELECT
            asset,
            purchase_date,
            due_date,
            COALESCE (lead(purchase_date) OVER (PARTITION BY asset ORDER BY purchase_date), due_date) AS next_date,
            CASE operation_type WHEN "buy" THEN quotas ELSE quotas * -1 END quotas,
            CASE operation_type WHEN "buy" THEN value ELSE value * -1 END value
        FROM fixed_income_operations
        WHERE asset like "%Tesouro Selic%"
    )
    SELECT asset, i.date, due_date, i.factor, quotas, value
    FROM index_series i
    INNER JOIN operations o
    ON i.date >= o.purchase_date and i.date < o.next_date 
    WHERE i.financial_index = "cdi"
    ORDER BY asset, i.date
""")

all_rows = cursor.fetchall()

previous_asset = ""
quota_balance = 0
balance = 0
compound_values = []
previous_purchase_value = 0

for (asset, date, due_date, factor, quota, value) in all_rows:
    if asset != previous_asset:
        previous_asset = asset
        quota_balance = quota
        balance = value
        previous_purchase_value = value
    else:
        if previous_purchase_value != value:
            balance = balance + value
            previous_purchase_value = value
            quota_balance += quota

        if quota_balance > 0:
            balance = balance * factor
        else:
            current_value = 0
             
    if (balance > 0):
        compound_values.append((previous_asset, due_date, date, 0.0, previous_purchase_value, balance, 0, balance))

In [13]:
cursor.executemany(
    """
    INSERT INTO fixed_income_daily_balance(asset, due_date, date, tax_rate, deposit_value, gross_value, tax_value, net_value)
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s)
    """,
    compound_values
)

### Insert fgts_daily_balance

In [14]:
cursor.execute("DELETE FROM fgts_daily_balance")

In [15]:
cursor.execute("""
    INSERT INTO fgts_daily_balance
    WITH fgts_by_company AS(
        SELECT date, company, max(balance) as balance
        FROM fgts_operations    
        GROUP BY date, company
    ), fgts_by_date AS (
        SELECT date, sum(balance) as balance, count(*) as companies
        FROM fgts_by_company    
        GROUP BY date
        HAVING count(*) > 1
    ), fgts_range AS (
        SELECT date, COALESCE(LEAD(date) OVER (ORDER BY date), "2099-01-01") as next_date, balance
        FROM fgts_by_date 
    )
    SELECT d.date, f.balance
    FROM dates d
    INNER JOIN fgts_range f
    ON d.date >= f.date and d.date < f.next_date

""")

In [16]:
cursor.execute("DELETE FROM daily_balance")

In [17]:
cursor.execute("""
    INSERT INTO daily_balance
    WITH daily_balances_cte AS
    (
        SELECT
            ticker,
            date,
            value,
            'equity' as type
        FROM variable_income_daily_balance
        WHERE date <= (SELECT max(date) FROM fixed_income_daily_balance)
        UNION
        SELECT
            asset as ticker,
            date,
            net_value as value,
            'fixed_income' as type
        FROM fixed_income_daily_balance
        WHERE date <= (SELECT max(date) FROM variable_income_daily_balance)
        UNION
        SELECT "FGTS", date, balance, "fixed_income"
        FROM fgts_daily_balance
        WHERE date <= (SELECT max(date) FROM fixed_income_daily_balance)
        AND date <= (SELECT max(date) FROM variable_income_daily_balance)
    )
    SELECT ticker, date, value, type
    FROM daily_balances_cte
""")

In [18]:
conn.commit()

In [19]:
conn.close()